## Metody analizy danych. Lab 2. Biblioteka numpy, część 2

In [1]:
import numpy as np
from timeit import timeit

## 1. Kopie i widoki

Wskazując na jakąś wybraną część istniejącej tablicy numpy czasem będziemy mieli do czynienia z **widokiem (ang. view)**, a czasem z **kopią (ang. copy)**. W niektórych przypadkach będziemy mieli wpływ na to jaki obiekt z tych dwóch typów zostanie zwrócony dla danej operacji, a w niektórych nie.

Jeżeli bliżej zapoznamy się z wewnętrzną organizacją tablic numpy to dowiemy się, że składają się one z ciągłej (ang. contiguous) przestrzeni pamięci uporządkowanej analogicznie do kolejności elementów w tablicy. Ten obszar jest również stałej wielkości ze względu na zawsze określany typ danych oraz liczebność tablicy w chwili jej tworzenia. Z racji tego, że pamięć w komputerze jest jednowymiarowa to przechowywanie tablic, które jak wiemy mogą być wielowymiarowe, odbywa się w krokach (ang. stride), których wielkość jest wyliczana na podstawie kształtu tablicy (ang. shape) oraz typu danych tablicy. Kroki są określane dla każdego wymiaru co pozwala na szybki odczyt i manipulację danych dla danej osi (wymiaru).

Bardziej szczegółowe informacje odnośnie wewnętrznej organizacji tablic numpy można znaleźć tu:
* https://numpy.org/doc/stable/dev/internals.html#numpy-internals
* https://www.labri.fr/perso/nrougier/from-python-to-numpy/#anatomy-of-an-array

In [ ]:
# inicjalizacja naszej tablicy do eksperymentów
A = np.arange(1, 5).reshape((2,2))

In [ ]:
# wprowadzenie kilku ważnych właściwości tablic numpy
# 1. base - wskazuje obiekt, z którym współdzieli pamięć lub None jeżeli jej nie współdzieli
# https://numpy.org/doc/2.2/reference/generated/numpy.ndarray.base.htmlA.base
A.base

array([1, 2, 3, 4])

In [ ]:
# wielkość pojedynczego elementu (w bajtach)
A.itemsize

8

In [ ]:
# liczba elementów w tablicy
A.size

5

In [ ]:
# co daje nam ilość zaalokowanej pamięci dla elementów tablicy
A.itemsize * A.size

40

In [ ]:
# która jest dostępna pod własnością nbytes również
A.nbytes

40

In [ ]:
# dwa wycinki, które wskazują na ten sam element, ale jak widać jeden z nich
# nie współdzieli pamięci z obiektem, z którego został wycięty, a drugi tak
# pierwszy to kopia, a drugi to widok
x = A[0][0] # 1
x_1 = A[:1, 0] # też 1
x_2 = A[[0]] # i to też 1
x.base, x_1.base, x_2.base

(None, array([1, 2, 3, 4]), None)

In [ ]:
# funkcje zaczerpnięte z https://tobiasraabe.github.io/post/numpy-views-vs-copies/
# dzięki nim łatwiej będzie nam interpretować czy dwie tablice numpy posiadają wspólną bazę
# co oznacza, że współdzielą pamięć (lub nie)

def get_data_base(x):
    base = x
    while isinstance(base.base, np.ndarray):
        base = base.base
    return base

def arrays_share_data(x, y):
    return get_data_base(x) is get_data_base(y)

In [ ]:
# na początek raz jeszcze sprawdźmy dwa wcześniejsze obiekty
arrays_share_data(x, A), arrays_share_data(x_1, A), arrays_share_data(x_2, A)

(False, True, False)

&#128165;&#128165;&#128165;&#128165;&#128165;  
Tu warto zapamiętać jedną regułę, którą zaprezentowano w poprzednim przykładzie. Zwykłe wycinki zwracają widok, a wycinki złożone (nazywane fancy indexing) zwracają kopię.

O fancy indexing można poczytać np. tu: https://jakevdp.github.io/PythonDataScienceHandbook/02.07-fancy-indexing.html

Poniżej zaprezentowano kilka dodatkowych przykładów.

In [ ]:
# ten wycinek jest widokiem
x_3 = A[0]
print(x_3)
arrays_share_data(x_3, A)

[1 2]


True

In [ ]:
# ten wycinek też jest widokiem, ale zwróć uwagę na shape
x_4 = A[:1] # pierwszy "wiersz"
print(x_4)
arrays_share_data(x_4, A)

[[1 2]]


True

In [ ]:
# wartość zwracana jest identyczna jak x_4, ale ten wycinek to kopia!
x_5 = A[:1, [0,1]]
print(x_5)
arrays_share_data(x_5, A)

[[1 2]]


False

In [ ]:
# wartość zwracana jest identyczna jak x_5, również kopia
x_6 = A[[0], ]
print(x_6)
arrays_share_data(x_6, A)

[[1 2]]


False

In [ ]:
# jeżeli określamy w formie listy zbiór wielu indeksów, do których chcemy
# się odwołać to otrzymamy kopię tablicy
arrays_share_data(A[[0,1]], A)

False

In [ ]:
# jeżeli jednak jest to postać slice, ellipsis (...) to otrzymamy widok
arrays_share_data(A[...], A)

True

In [ ]:
# mimo, że x_7 oraz x_8 wygląda niemal identycznie w zapisie (krotka vs. lista)
# to wynik przez nie zwracany jest różny jeżeli chodzi o shape
x_7 = A[(0,)]
print(x_7)
arrays_share_data(x_7, A)

[1 2]


True

In [ ]:
x_8 = A[[0,]]
print(x_8)
arrays_share_data(x_8, A)

[[1 2]]


False

In [ ]:
# a to dlatego, że ellipsis jest pomijany i oba poniższe wycinki są równoważne
x_9 = A[0,]
print(x_9, A[0])
arrays_share_data(x_9, A),arrays_share_data(A[0], A)

[1 2] [1 2]


(True, True)

Ciekawostką może być fakt, że zmiana kształtu tablicy nie musi implikować jakichkolwiek zmian jej elementów, chociaż za dokumentacją w https://numpy.org/doc/stable/reference/generated/numpy.reshape.html nie jest to gwarantowane.

Niektóre zmiany parametrów tablicy numpy powodują zmianę jej metadanych (parametrów) określających jak wyglądają własności ją opisujące, a nie położenie samych danych w pamięci.

Poniżej przykład z naszą tablicą A.

In [ ]:
B = A.reshape((4,))
B

array([1, 2, 3, 4])

In [ ]:
arrays_share_data(B, A)

True

In [ ]:
B.base is A.base

True

In [ ]:
B.base, A.base

(array([1, 2, 3, 4]), array([1, 2, 3, 4]))

In [ ]:
A, B

(array([[1, 2],
        [3, 4]]),
 array([1, 2, 3, 4]))

Nie trzeba tu chyba wyjaśniac, że kopie będą alokowały kolejne obszary pamięci oraz zajmą więcej czasu procesora w odróżnieniu od widoków.

Czy to znaczy, że widoki są dobre, a kopie złe? Absolutnie nie. Każde z nich ma swoje zastosowania, ważne, żeby używać ich świadomie.

Poniżej kilka przykładów ich zastosowania.

In [ ]:
# 1. widok vs kopia i wstawienie danych
Z = np.zeros(9)
Z_view = Z[:3]
Z_view[...] = 1

In [ ]:
Z

array([1., 1., 1., 0., 0., 0., 0., 0., 0.])

In [ ]:
Z_view

array([1., 1., 1.])

In [ ]:
Z = np.zeros(9)
Z_view = Z[[1,2,3]]
Z_view[...] = 1

In [ ]:
Z

array([0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [ ]:
Z_view

array([1., 1., 1.])

In [ ]:
# 2. obliczenia na wybranej części tablicy
C = np.arange(100_000_000).reshape((10_000,10_000))

In [ ]:
C.nbytes

800000000

In [ ]:
C_1 = C[:5000,:5000]

In [ ]:
# upewniamy się, że to widok
arrays_share_data(C_1, C)

True

In [ ]:
C_1.shape, C_1.size, C_1.mean()

((5000, 5000), 25000000, np.float64(24997499.5))

In [ ]:
# mamy jakąś listę indeksów, które nas interesują w tablicy
idx = range(5000)
C_2 = C[idx, :5000]

In [ ]:
# a teraz mamy kopię
arrays_share_data(C_2, C)

False

In [ ]:
# ten atrybut pozwala przyglądnąć się niektórym atrybutom tablicy, związanym z zaalokowaną dla niej pamięcią
# C_CONTIGUOUS oznacza ciągłość pamięci w stylu języka C
# F_CONTIGUOUS oznacza ciągłość pamięci w stylu języka Fortran
# OWNDATA określa czy tablica współdzieli dane z inną (False oznacza, że jej base to odwołanie do innej tablicy) lub czy jest ich właścicielem (True)
# WRITEABLE określa czy dane w tablicy mogą być nadpisane (domyślnie True), można ustawić tę flagę na True i zablokować możliwość ich nadpisania.
# Jest to możliwe również dla widoków utworzonych z tej tablicy, mimo, że oryginalna tablica pozwala na nadpisanie danych w tym obszarze.
# ALIGNED czy dane są ułożone odpowiednio dla sprzętu (ang. hardware)
# WRITEBACKIFCOPY czy jest kopią innej tablicy (za dokumentacją), ale ten koncept jest nieco bardziej skomplikowany i odsyłam czytelnika do dokumentacji
C.flags

  C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : False
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False

In [ ]:
C_1.flags

  C_CONTIGUOUS : False
  F_CONTIGUOUS : False
  OWNDATA : False
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False

In [ ]:
C_2.flags

  C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False

In [ ]:
# innym sposobem na sprawdzenie czy tablice współdzielą dane jest sprawdzenie czy dany element, który wycięliśmy z oryginału
# (zakładając, że tak iż powstał widok), ma taki sam identyfikator pamięci jak adekwatny element w tablicy oryginalnej
# Dlaczego indeksowanie w drugim przypadku odbywa się po jednym wymiarze - odpowiedż w komórce poniżej - base jest widoczne jako tablica
# jednowymiarowa
id(C[0][0]) == id(C_1.base[0])

True

In [ ]:
C.shape, C_1.base.shape, C_1.shape

((10000, 10000), (100000000,), (5000, 5000))

In [ ]:
id(C), id(C_1.base)

(2707294963536, 2707315342576)

In [ ]:
C_2.shape, C_2.size, C_2.mean()

((5000, 5000), 25000000, np.float64(24997499.5))

**I jeszcze jeden przykład z widokami i flagami**

In [ ]:
# mamy takie dwie tablice
D = np.arange(25)
E = D[:5]

In [ ]:
# widzimy ciągłość danych w obu  (zazwyczaj dla tablic jednowymiarowych)
D.flags, E.flags

(  C_CONTIGUOUS : True
   F_CONTIGUOUS : True
   OWNDATA : True
   WRITEABLE : True
   ALIGNED : True
   WRITEBACKIFCOPY : False,
   C_CONTIGUOUS : True
   F_CONTIGUOUS : True
   OWNDATA : False
   WRITEABLE : True
   ALIGNED : True
   WRITEBACKIFCOPY : False)

In [ ]:
# identyfikatory również nam się zgadzają
id(D), id(E.base)

(2707311220784, 2707311220784)

In [ ]:
# zmieniamy kształt tablicy bazowej, nadpisujemy zmienną D
D = D.reshape((5,5))

In [ ]:
# identyfikatory już nie wskazują na ten sam obiekt
id(D), id(E.base)

(2707311223568, 2707311220784)

In [ ]:
# ale baza dla E wygląda znajomo
E.base

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24])

In [ ]:
# tablice jednak współdzielą dane
arrays_share_data(E, D)

True

In [ ]:
# sprawdzimy id lokalizacji pierwszego elementu obu tablic
id(D[0][0]), id(E[0])

(2707294120912, 2707294120912)

In [ ]:
# identyfikator się zgadza, ale flagi już zawierają pewne różnice
D.flags, E.base.flags, E.flags

(  C_CONTIGUOUS : True
   F_CONTIGUOUS : False
   OWNDATA : False
   WRITEABLE : True
   ALIGNED : True
   WRITEBACKIFCOPY : False,
   C_CONTIGUOUS : True
   F_CONTIGUOUS : True
   OWNDATA : True
   WRITEABLE : True
   ALIGNED : True
   WRITEBACKIFCOPY : False,
   C_CONTIGUOUS : True
   F_CONTIGUOUS : True
   OWNDATA : False
   WRITEABLE : True
   ALIGNED : True
   WRITEBACKIFCOPY : False)

In [ ]:
# dzieje się tak z powodu funkcji reshape(), która zwraca nowy widok danych z tablicy, której wymiary zmienia jeżeli tylko się da
# lub kopię jeżeli to konieczne
# To oznacza, że numpy odwołuje się do tablicy bazowej stworzonej przez wywołanie np.arange(25), która stworzyła tablicę jednowymiarową
# składającą się z kolejnych 25 elementów poczynając od wartości 0

# jak sprawdzimy poniższe to otrzymamy potwierdzenie
id(D.base), id(E.base)

(2707311220784, 2707311220784)

### 1.1 Pomiary wydajności

Wykorzystamy moduł `timeit` do wykonania poamiarów średniej prędkości wykonania obliczeń wybranych fragmentów kodu.

In [ ]:
# do poamiarów wykorzystamy nasze tablice C, C_1 oraz C_2
# Tu zmierzymy czas niezbędny do odwołania się do obszaru o wymiarach (5000,5000) z tablicy bazowej C
# Wykorzystujemy wycinki, ale pierwszy to widok, a drugi to kopia

setup = """
import numpy as np
C = np.arange(100_000_000).reshape((10_000,10_000))
"""

stmt_1 = """
C_1 = C[:5000,:5000]
"""

stmt_2 = """
idx = range(5000)
C_2 = C[idx, :5000]
"""

timeit(stmt_1, setup=setup, number=1000), timeit(stmt_2, setup=setup, number=1000)

(6.13001175224781e-05, 11.454770299838856)

Jak widać samo utworzenie zmiennej z dostępem do wskazanych danych jest o kilka rzędów szybsze w przypadku widoków niż kopii co brzmii jak coś oczywistego biorąc pod uwagę koszt dostępu do już zarezerwowanego obszaru pamięci, a jego ponownego przypisania (tutaj dla wielu wartości).
Postaramy się zmierzyć jeszcze inne przykładowe operacje.

In [ ]:
# teraz policzymy czas wyliczenia średniej dla tych samych obszarów

setup = """
import numpy as np
C = np.arange(100_000_000).reshape((10_000,10_000))
"""

stmt_1 = """
C[:5000,:5000].mean()
"""

stmt_2 = """
idx = range(5000)
C[idx, :5000].mean()
"""

timeit(stmt_1, setup=setup, number=100), timeit(stmt_2, setup=setup, number=100)

(22.169761300086975, 83.69246080005541)

W powyższym przykładzie wynik jest już bardziej zbliżony, ale wciąż pierwszy przypadek wykonuje się szybciej, tu około 4x. Pamiętajmy, że oprócz krótszego czasu mamy tu również oszczędność pamięci, co dla większych zbiorów danych ma znaczenie, szczególnie w sytuacji kiedy zbliżamy się lub przekraczamy limit pamięci RAM, co w kolejnej fazie powoduje wykorzystanie pamięci dyskowej, które jest zazwyczaj znacznie wolniejsza od pamięci RAM.

**Funkcja np.view**

Funkcja np.view pozwala na stworzenie widoku na oryginalną tablicę numpy, której obiekty możemy "oglądać" w innym typie niż w oryginale. Poniżej przykład.

In [ ]:
D = np.arange(10)

In [ ]:
D

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [ ]:
E = D.view(np.int32)

In [ ]:
E.base

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [ ]:
D.dtype, E.dtype

(dtype('int64'), dtype('int32'))

In [ ]:
D.nbytes, E.nbytes

(80, 80)

In [ ]:
E.size

20

In [ ]:
E

array([0, 0, 1, 0, 2, 0, 3, 0, 4, 0, 5, 0, 6, 0, 7, 0, 8, 0, 9, 0],
      dtype=int32)

## 2. Wybrane funkcje numpy.

### 2.1 Zmiana kształtu tablicy

**numpy.reshape()**

In [ ]:
A = np.ones(16)

In [ ]:
# 1. np.reshape już znamy
A.reshape((4,4))

array([[1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.]])

In [ ]:
# lub tak
A.reshape(4,4)

array([[1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.]])

In [ ]:
A.reshape((2,2,4))

array([[[1., 1., 1., 1.],
        [1., 1., 1., 1.]],

       [[1., 1., 1., 1.],
        [1., 1., 1., 1.]]])

In [ ]:
# możemy zmienić domyśle zachowanie, które zwraca widok, chcąc zwrócić kopię
B = A.reshape((4,4), copy=True)
print(B)
arrays_share_data(B,A)

[[1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]]


False

In [ ]:
# reshape pozwala nam na przekazanie "nieznanej" wartości dla tylko jednej z osi poprzez podanie wartości -1
# spłaszczenie
print(B.reshape(-1))

print(B.reshape(2,-1))
print(B.reshape(2,2,-1))

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[[1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1.]]
[[[1. 1. 1. 1.]
  [1. 1. 1. 1.]]

 [[1. 1. 1. 1.]
  [1. 1. 1. 1.]]]


**numpy.ravel()**

In [ ]:
# zwraca tablicę jednowymiarową stworzoną ze wskazanej tablicy bazowej
np.ravel(B)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [ ]:
# większość funkcji możemy również wywołać z poziomu instancji obiektu np.ndarray
B.ravel()

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [ ]:
# widok czy kopia?
arrays_share_data(B, B.ravel()) # widok

True

**numpy.ndarray.flatten()**

In [ ]:
# dzdiała podobnie do ravel, ale jest to metoda klasy ndarray, więc jej wywołanie odbywa się z poziomu obiektu
B.flatten()

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [ ]:
# widok czy kopia?
arrays_share_data(B, B.flatten()) # kopia

False

**numpy.ndarray.flat**

In [ ]:
# zwraca iterator pozwalający przychodzić element po elemencie dla danej tablicy, tak jakby była jednowymiarowa
B.flat

Istnieją jeszcze inne możliwości takiej jak użycie `rot90`, `flip`, `fliplr`, `flipud`, ale pozostawiam je do eksploracji dla czytelnika.

### 2.2 Łączenie tablic

In [ ]:
A = np.arange(1, 6)
B = np.arange(6, 11)
A, B

(array([1, 2, 3, 4, 5]), array([ 6,  7,  8,  9, 10]))

In [ ]:
# chyba coś poszło nie tak?
# Na pewno działa dla list, ale dla np.ndarray operator __add__ (inaczej +) działa inaczej
# wykonuje operację element-wise - czyli element po elemencie
A + B

array([ 7,  9, 11, 13, 15])

In [ ]:
# to oznacza, że poniższe nie zadziała - wymiary się nie zgadzają
A + np.arange(3)

ValueError: operands could not be broadcast together with shapes (5,) (3,) 

In [ ]:
# do łączenia tablic musimy wykorzystać inne metody, tu concatenate
np.concatenate((A,B))

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

In [ ]:
# ta funkcja przyjmuje również argument o nazwie axis, który jest dość powszechny w przypadku biblioteki numpy i określa oś
# dla której operacja ma zostać wykonana (pojawi się jeszcze wielokrotnie)
C = np.arange(4).reshape(2,2)
D = np.arange(4,8).reshape(2,2)
print(np.concatenate((C,D), axis=1), np.concatenate((C,D), axis=1).shape)
print(np.concatenate((C,D), axis=0), np.concatenate((C,D), axis=0).shape)

[[0 1 4 5]
 [2 3 6 7]] (2, 4)
[[0 1]
 [2 3]
 [4 5]
 [6 7]] (4, 2)


In [ ]:
# np.stack również łączy ze sobą tablice, ale tworzy nowy wymiar w tablicy wynikowej
# domyślnie axis=0
print(np.stack((C,D)))
print(np.stack((C,D)).shape)

[[[0 1]
  [2 3]]

 [[4 5]
  [6 7]]]
(2, 2, 2)


In [ ]:
# a dal axis=1 mamy
print(np.stack((C,D), axis=1))
print(np.stack((C,D), axis=1).shape)

[[[0 1]
  [4 5]]

 [[2 3]
  [6 7]]]
(2, 2, 2)


In [ ]:
# są też funkcje, które stakują tablice wierszami
print(np.hstack((C,D)))
print(np.hstack((C,D)).shape)

[[0 1 4 5]
 [2 3 6 7]]
(2, 4)


In [ ]:
# oraz kolumnami
print(np.vstack((C,D)))
print(np.vstack((C,D)).shape)

[[0 1]
 [2 3]
 [4 5]
 [6 7]]
(4, 2)


In [ ]:
# możemy również stackować tablice różnych wymiarów, ale są oczywiście ograniczenia
E = np.vstack((C,D))
np.vstack((E,C))

array([[0, 1],
       [2, 3],
       [4, 5],
       [6, 7],
       [0, 1],
       [2, 3]])

In [ ]:
F = np.arange(8).reshape(4,2)
np.vstack((F,C))
# np.hstack((F,C)) # błąd!

array([[0, 1],
       [2, 3],
       [4, 5],
       [6, 7],
       [0, 1],
       [2, 3]])

## Zadania

**Zadanie 1**

Stwórz tablicę numpy o wymiarach (4,4) wypełnioną wartościami od 2 do 32 z krokiem 2 i przypisz ją do zmiennej.
Wyświetl jej typ danch, liczbę elementów, wielkość pojedynczego elementu oraz ilość zaalokowanej pamięci.

**Zadanie 2**

Zwróć tablicę z zadania 1 jako widok.
Zmień jej typ danych (możesz to zrobić np. funkcją `astype()`) na np.uint8.
Sprawdź teraz czy tablica jest widokiem czy kopią tablicy oryginalnej. Wyświetl jej liczbę elementów, wielkość pojedynczego elementu oraz ilość zaalokowanej pamięci.

**Zadanie 3**

Wytnij z tablicy z zadania 1 obszar 2 x 2 "ze środka" tablicy (wartości 12,14, 20,22) na dwa sposoby:
* jeden jako widok i przypisz do zmiennej
* drugi jako kopia i przypisz do zmiennej

Posługując się przykładami z zajęć sprawdź czy baza tych dwóch tablic jest różna (powinna być).

**Zadanie 4**

Stwórz tablicę 4x4, która zawiera wartości 1,2,3,4 w każdym wierszu.
Następnie wykorzystując funkcje łączenia tablic utwórz tablicę połączoną z powyższej tablicy i jej wersji lustrzanej w poziomie (spróbuj użyć wycinków).

**Zadanie 5**

Utwórz tablicę o nazwie `Z_5` o wymiarach 2x2 wypełnioną wartościami 1.
Wykorzystując funkcje łączenia tabel uwtórz tablicę, która finalnie będzie tablicą składającą się z 4-ech tablic `Z_5`, ale tak aby finalnie była to tablica 4 x 4 podobna do poniższej:

1,1,2,2  
1,1,2,2  
3,3,4,4  
3,3,4,4  

In [8]:
tab = np.arange(2, 34, 2).reshape(4, 4)
print(tab.dtype)
print(tab.size)
print(tab.itemsize)
print(tab.nbytes)

int64
16
8
128


In [9]:
tab_view = tab.astype(np.uint8)
print(tab_view.base is tab)
print(tab_view.size)
print(tab_view.itemsize)
print(tab_view.nbytes)

False
16
1
16


In [10]:
tab_view = tab[1:3, 1:3]
tab_copy = tab[1:3, 1:3].copy()
print(tab_view.base is tab)
print(tab_copy.base is tab)

False
False


In [11]:
tab_4x4 = np.tile(np.array([1, 2, 3, 4]), (4, 1))
tab_mirror = tab_4x4[:, ::-1]
tab_combined = np.hstack((tab_4x4, tab_mirror))
print(tab_combined)

[[1 2 3 4 4 3 2 1]
 [1 2 3 4 4 3 2 1]
 [1 2 3 4 4 3 2 1]
 [1 2 3 4 4 3 2 1]]


In [13]:
Z_5 = np.ones((2, 2), dtype=int)
Z_5_2 = Z_5 * 2
Z_5_3 = Z_5 * 3
Z_5_4 = Z_5 * 4
tab_final = np.block([[Z_5, Z_5_2], [Z_5_3, Z_5_4]])
print(tab_final)

[[1 1 2 2]
 [1 1 2 2]
 [3 3 4 4]
 [3 3 4 4]]
